In [6]:
import pandas as pd
import os
import shutil
import chardet

def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read(5000))  # Lire un échantillon plus grand pour améliorer la précision
    return result['encoding']

def read_csv_with_encoding(file_path):
    encoding_attempts = ['utf-8', 'cp1252', detect_encoding(file_path)]
    for encoding in encoding_attempts:
        try:
            df = pd.read_csv(file_path, encoding=encoding, sep=";", on_bad_lines='skip')
            print(f"Le fichier a été lu avec succès en utilisant l'encodage {encoding}")
            return df
        except UnicodeDecodeError:
            continue
    raise ValueError(f"Tous les encodages ont échoué pour le fichier {file_path}")

folder_path = '/home/patea/code/Pateaanania/historique_lit'
processed_folder = '/home/patea/code/Pateaanania/processed'
empty_files_folder = '/home/patea/code/Pateaanania/empty_files'

os.makedirs(folder_path, exist_ok=True)
os.makedirs(processed_folder, exist_ok=True)
os.makedirs(empty_files_folder, exist_ok=True)

dataframes = []

if os.listdir(folder_path):
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if file_name.endswith('.csv') and os.stat(file_path).st_size > 0:
            try:
                df = read_csv_with_encoding(file_path)
                if not df.empty:
                    dataframes.append(df)
                shutil.move(file_path, os.path.join(processed_folder, file_name))
            except Exception as e:
                print(f"Erreur lors de la lecture du fichier {file_name}: {e}")
                shutil.move(file_path, os.path.join(empty_files_folder, file_name))
else:
    print("Le dossier est vide.")

if dataframes:
    final_dataframe = pd.concat(dataframes, ignore_index=True)
    output_parquet = 'resultat_concatenation_avec_date_heure.parquet'
    final_dataframe.to_parquet(output_parquet, index=False)
    print(f"Le fichier Parquet a été créé avec succès : {output_parquet}")
else:
    print("Aucun fichier CSV valide trouvé ou le dossier est vide.")


Le dossier est vide.
Aucun fichier CSV valide trouvé ou le dossier est vide.
